# Question-answering engine

In [ ]:
!pip install datasets transformers

## Compress all into functions for easier use

In [2]:
def prepare_train_features(examples, tokenizer):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    max_length = 384 # The maximum length of a feature (question and context)
    doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
    pad_on_right = tokenizer.padding_side == "right"

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [3]:
def prepare_validation_features(examples, tokenizer):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    max_length = 384 # The maximum length of a feature (question and context)
    doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
    pad_on_right = tokenizer.padding_side == "right"
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [4]:
def postprocess_qa_predictions(examples, tokenizer, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer
        
    return predictions

In [5]:
def fine_tune_model(checkpoint, datasets, saved_file=None, train=True):
  """
  Prepare dataset and fine-tune a model using pre-trained values (checkpoint).

  Input:
    * Checkpoint: Checkpoint of a Question_Answering model from the Huggingface library.
    * datasets: Dict of key "train" and "validation" and value two datasets.
    * save_file: Path to model save file

  Return:
    A fine-tuned trainer object.
  """

  # Hyperparameters
  batch_size = 16

  # Create first tokenizer
  tokenizer = AutoTokenizer.from_pretrained(basic_checkpoint)
  
  # Tokenize datasets
  train_ds = datasets['train'].map(lambda x: prepare_train_features(x, tokenizer), batched=True, remove_columns=datasets['train'].column_names)
  val_ds = datasets['validation'].map(lambda x: prepare_train_features(x, tokenizer), batched=True, remove_columns=datasets['train'].column_names)

  # Create model
  if saved_file is None:
    model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)
  else:
    model = AutoModelForQuestionAnswering.from_pretrained(saved_file, local_files_only=True)
    model.to('cuda')

  # Model parameters
  model_name = checkpoint.split("/")[-1]
  args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
  )

  # Create datacollator
  data_collator = default_data_collator

  # Create trainer
  trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
  )

  # Train model
  if train:
    trainer.train()

  # Save model
  if train:
    trainer.save_model(model_name + "_save")

  return trainer, tokenizer

In [6]:
def evaluate_model(trainer, tokenizer):
  # Hyperparameters
  n_best_size = 20
  max_answer_length = 30
  
  # Prepare validation datasets
  validation_features = datasets["validation"].map(
    lambda x : prepare_validation_features(x, tokenizer),
    batched=True,
    remove_columns=datasets["validation"].column_names
  )

  # Calculate raw predictions
  raw_predictions = trainer.predict(validation_features)

  # Add usefull columns
  validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

  # Retrieve features
  examples = datasets["validation"]
  features = validation_features

  example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
  features_per_example = collections.defaultdict(list)
  for i, feature in enumerate(features):
      features_per_example[example_id_to_index[feature["example_id"]]].append(i)

  # Calculate final predictions
  final_predictions = postprocess_qa_predictions(datasets["validation"], tokenizer, validation_features, raw_predictions.predictions)

  # Get metric for evaluation
  #metric = load_metric("squad_v2")

  # Evaluate model
  formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

  references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
  return formatted_predictions, references
  #return metric.compute(predictions=formatted_predictions, references=references)

## Full code

In [8]:
# Import
import transformers
import collections
import numpy as np
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
from transformers import default_data_collator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# Get model checkpoint
basic_checkpoint = "distilbert-base-uncased"
teacher_checkpoint = "mvonwyl/distilbert-base-uncased-finetuned-squad2"

# Get dataset
datasets_ = load_dataset("squad_v2", split=['train[:20000]', 'validation[:5000]'])

# Get metric for evaluation
metric = load_metric("squad_v2")

# Create dataset dictionnary
datasets = {}
datasets["train"] = datasets_[0]
datasets["validation"] = datasets_[1]

# Fine tune our basic model

basic_trainer, basic_tokenizer = fine_tune_model(basic_checkpoint, datasets, train=True)
teacher_trainer, teacher_tokenizer = fine_tune_model(teacher_checkpoint, datasets, train=False)

# Evaluate models
basic_prediction, basic_reference = evaluate_model(basic_trainer, basic_tokenizer)
teacher_prediction, teacher_reference = evaluate_model(teacher_trainer, teacher_tokenizer)

# Print metrics
print(metric.compute(predictions=basic_prediction, references=basic_reference))
print(metric.compute(predictions=teacher_prediction, references=teacher_reference))

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-f97256d5920f643d.arrow
The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 5225
  Batch size = 16


Post-processing 5000 example predictions split into 5225 features.


  0%|          | 0/5000 [00:00<?, ?it/s]

{'exact': 45.72, 'f1': 49.77346504131027, 'total': 5000, 'HasAns_exact': 51.21555915721232, 'HasAns_f1': 59.427603406220214, 'HasAns_total': 2468, 'NoAns_exact': 40.363349131121645, 'NoAns_f1': 40.363349131121645, 'NoAns_total': 2532, 'best_exact': 50.9, 'best_exact_thresh': 0.0, 'best_f1': 53.1008787674189, 'best_f1_thresh': 0.0}
{'exact': 65.3, 'f1': 68.988572537942, 'total': 5000, 'HasAns_exact': 67.09886547811993, 'HasAns_f1': 74.5716623540156, 'HasAns_total': 2468, 'NoAns_exact': 63.54660347551343, 'NoAns_f1': 63.54660347551343, 'NoAns_total': 2532, 'best_exact': 65.36, 'best_exact_thresh': 0.0, 'best_f1': 69.00857253794186, 'best_f1_thresh': 0.0}


In [9]:
def find_paragraph(val_set, id):
  for element in val_set:
    if element['id'] == id:
      return element

In [10]:
def show_false_pred(prediction, reference, validation_set, number_pred=2, skip=0):
  length = len(reference)
  nb_pred = number_pred

  for i in range(length):
    if prediction[i]['prediction_text'] not in reference[i]['answers']['text']:

      if skip > 0:
        skip -= 1
        continue
  
      # Print answer
      para = find_paragraph(validation_set, prediction[i]['id'])
      print(f"Wrong answer number #{number_pred - nb_pred + 1}")
      print(prediction[i])
      print(reference[i])
      print(para, end='\n\n')
      nb_pred -= 1
    
    if nb_pred == 0:
      break

In [11]:
show_false_pred(basic_prediction, basic_reference, datasets['validation'], skip=20)

Wrong answer number #1
{'id': '5ad3ae14604f3c001a3fec3a', 'prediction_text': 'pagii', 'no_answer_probability': 0.0}
{'id': '5ad3ae14604f3c001a3fec3a', 'answers': {'text': [], 'answer_start': []}}
{'id': '5ad3ae14604f3c001a3fec3a', 'title': 'Normans', 'context': 'Before Rollo\'s arrival, its populations did not differ from Picardy or the Île-de-France, which were considered "Frankish". Earlier Viking settlers had begun arriving in the 880s, but were divided between colonies in the east (Roumois and Pays de Caux) around the low Seine valley and in the west in the Cotentin Peninsula, and were separated by traditional pagii, where the population remained about the same with almost no foreign settlers. Rollo\'s contingents who raided and ultimately settled Normandy and parts of the Atlantic coast included Danes, Norwegians, Norse–Gaels, Orkney Vikings, possibly Swedes, and Anglo-Danes from the English Danelaw under Norse control.', 'question': 'What Viking groups were conquered by Rollo?', 

For the first false prediction here, for the question **What Viking groups were conquered by Rollo?**, the basic_model predicted that the answer was **pagii** when none was include in the text. We can understand its mistake because the waited answer was about an ethnicity and the predicted answer is a lost word for country in latin.

For the second prediction, the question was **What was the Norman religion?** and the model did not find an answer in the text but **Catholicism** was in foundable. It is certainly due to a lack of training, because as we will see below, the best model found something (even if not right) for this question.

In [12]:
show_false_pred(teacher_prediction, teacher_reference, datasets['validation'], skip=20)

Wrong answer number #1
{'id': '56dde27d9a695914005b9651', 'prediction_text': 'Norse', 'no_answer_probability': 0.0}
{'id': '56dde27d9a695914005b9651', 'answers': {'text': ['Catholicism', 'Catholicism', 'Catholicism'], 'answer_start': [121, 121, 121]}}
{'id': '56dde27d9a695914005b9651', 'title': 'Normans', 'context': 'The descendants of Rollo\'s Vikings and their Frankish wives would replace the Norse religion and Old Norse language with Catholicism (Christianity) and the Gallo-Romance language of the local people, blending their maternal Frankish heritage with Old Norse traditions and customs to synthesize a unique "Norman" culture in the north of France. The Norman language was forged by the adoption of the indigenous langue d\'oïl branch of Romance by a Norse-speaking ruling class, and it developed into the regional language that survives today.', 'question': 'What was the Norman religion?', 'answers': {'text': ['Catholicism', 'Catholicism', 'Catholicism'], 'answer_start': [121, 121,

For the first false prediction, the question was **What was the Norman religion?**, the teacher model predicted that the answer was **Norse** which refers to the replaced religion and Catholicism was the right answer. We can see that the model grasped the sense of religion for those words, but mistaked on which word so it may lack training for getting the context right.

For the second prediction, the question was **What was replace with the Norse religion?** and the model did find an answer **Catholicism** when none was in the texte. It shows once again that the model gets the religion sense of those word but again, reverse the question context and predict that Norse is the new religion and Catholicism the replaced one.